Hi Jan, while searching for the bug for my PQK kernel DE, I came across something strange. I dont know if it is a matter of different definitions or something else. 


If one applies an $R_x(x)$ rotation in a circuit and calculates the expectation value with regards to $X,Y$ and $Z$. One finds that (see derivation below):

$<X \rho> =  0$ 


$<Y\rho> = sin(x)$ 

$<Z\rho> = cos(x)$ 


If one uses squlearn, one finds that the sign with regards to Y is changed:

$<X \rho> =  0$ 


$<Y\rho> = -sin(x)$ 

$<Z\rho> = cos(x)$  

Do you have an explanation why?

Derivation using sympy

In [58]:
import sympy as sp
#also https://arxiv.org/pdf/2206.06686 equation 3 and 4
x = sp.symbols("x", real=True)
RX = sp.Matrix([[sp.cos(x/2), 1j*sp.sin(x/2)], [1j*sp.sin(x/2), sp.cos(x/2)]])

X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -1j], [1j, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])

ket0 = sp.Matrix([[1], [0]])
bra0 = ket0.T

rhox = RX @ ket0 @ bra0 @ RX.H

value = 0.12
print(f"f(x) = {sp.trace(X@rhox).simplify()}, {sp.trace(Y@rhox).simplify()}, {sp.trace(Z@rhox).simplify()}")
print("f(0.5)", [f.subs(x, value).evalf() for f in [sp.trace(X@rhox), sp.trace(Y@rhox), sp.trace(Z@rhox)]])


f(x) = 0, 1.0*sin(x), 1.0*cos(x)
f(0.5) [0, 0.119712207288919, 0.992808635853866]


Experiment squlearn

In [59]:
import numpy as np
from squlearn.observables import  SinglePauli
from squlearn.encoding_circuit import LayeredEncodingCircuit
from squlearn.qnn.lowlevel_qnn import LowLevelQNN
from squlearn import Executor

def Separable_rx(num_qubits, num_layers):
    """
    Separable_rx(num_qubits, num_layers)
    Returns a circuit that is similar to the one used in IQP.
    """
    fmap = LayeredEncodingCircuit(num_qubits=num_qubits, num_features=num_qubits)
    for layer in range(num_layers):
        fmap.Rx("x")
    return fmap

num_qubits = 1
x_array = np.array([[value]])

circuit = Separable_rx(1, 1)
observable = [SinglePauli(num_qubits, 0, "X"), SinglePauli(num_qubits, 0, "Y"), SinglePauli(num_qubits, 0, "Z")]
qnn_pennylane_statevector = LowLevelQNN(circuit, observable, Executor("pennylane"))

param = []
param_obs = []

print("Pennylane statevector ")
print("f\n", qnn_pennylane_statevector.evaluate(x_array, param, param_obs, "f")["f"])

Pennylane statevector 
f
 [[ 0.         -0.11971221  0.99280864]]
